In [13]:
import numpy as np
import pandas as pd

from src.constant import DATA_DIR, MAIN_DIR
from src.database import DB
from src.database.queries import *
from src.instance.InstanceList import InstanceList
from src.instance.TSP_Instance import TSP_from_index_file
from src.solver.TSP_LKH_Solver import TSP_LKH_Solver

In [14]:
generators = [
    "cluster_netgen",
    "compression",
    "expansion",
    "explosion",
    "grid",
    "cluster",
    "implosion",
    "linearprojection",
    "rotation",
    "uniform_portgen",
]

test_instances = TSP_from_index_file(filepath=DATA_DIR / "TSP" / "TEST_600" / "index.json")

id_to_generator_name = {
    instance.id(): instance.filepath.parts[-2] for instance in test_instances
}

id_to_name = {
    instance.id(): instance.filepath.parts[-1] for instance in test_instances
}

In [19]:
def agg_prefix(prefix):

    frames = []

    def agg_cost(x):
        return sum(x == TSP_LKH_Solver.MAX_COST)
        

    for idx, db_path in enumerate((MAIN_DIR / "_archive" / "experiments" / "2025-01-27" / "600" / "database").glob(f"{prefix}-*.db")):
        db = DB(db_path)
        results = pd.read_sql_query("SELECT * FROM results", db._conn)
        series = (
            results.loc[results["prefix"].str.startswith("test")]
            .groupby(["instance_id", "prefix"])["cost"]
            .min()
            .reset_index()
            .groupby("instance_id")["cost"]
            .agg(agg_cost)
        )
        frames.append(series)
        

    df = (
        pd.concat(frames, axis=1)
        .sum(axis=1)
        .rename(prefix)
        .to_frame()
    )

    df["generator"] = df.index.map(id_to_generator_name)
    df["name"] = df.index.map(id_to_name)
    df = df.pivot_table(index="generator", columns="name", values=prefix).loc[generators, :]
    return df

n = 250
plain = agg_prefix(f"run-plain-{n}")
sur25 = agg_prefix(f"run-{n}-sur-25")
sur50 = agg_prefix(f"run-{n}-sur-50")
sur75 = agg_prefix(f"run-{n}-sur-75")

plain = plain.assign(generator=f"plain-{n}").set_index("generator", append=True)
sur25 = sur25.assign(generator="surrogate-25").set_index("generator", append=True)
sur50 = sur50.assign(generator="surrogate-50").set_index("generator", append=True)
sur75 = sur75.assign(generator="surrogate-75").set_index("generator", append=True)

In [20]:
df = pd.concat([plain, sur25, sur50, sur75], axis=0).sort_index(level=0).loc[generators]   

In [21]:
df.style.background_gradient(cmap="Reds", vmin=0, vmax=100).format("{:.0f}")

In [22]:
plain_total_mean = plain.mean(axis=1).mean(axis=0) / 1000
sur25_total_mean = sur25.mean(axis=1).mean(axis=0) / 1000
sur50_total_mean = sur50.mean(axis=1).mean(axis=0) / 1000
sur75_total_mean = sur75.mean(axis=1).mean(axis=0) / 1000
print(f"{plain_total_mean=:.5f}")
print(f"{sur25_total_mean=:.5f}")
print(f"{sur50_total_mean=:.5f}")
print(f"{sur75_total_mean=:.5f}")

plain_total_mean=0.00590
sur25_total_mean=0.00326
sur50_total_mean=0.00830
sur75_total_mean=0.00366


In [23]:
# 
df.to_excel("tmp.xlsx")